In [2]:
index_file = open("/Users/leftstories/Desktop/10k/company.idx").readlines()
print(index_file[:10])

find_list = []
item = 0
line = 0

while item < 100:
    i = index_file[line]
    loc1 = i.find('10-K')
    loc2 = i.find("NT 10-K") 
    loc3 = i.find("10-K/A")
    if (loc2 == -1) and (loc1 != -1) and (loc3 == -1):
       find_list.append(i)
    line+=1
    item = len(find_list)



['Description:           Master Index of EDGAR Dissemination Feed by Company Name\n', 'Last Data Received:    March 31, 2021\n', 'Comments:              webmaster@sec.gov\n', 'Anonymous FTP:         ftp://ftp.sec.gov/edgar/\n', ' \n', ' \n', ' \n', ' \n', 'Company Name                                                  Form Type   CIK         Date Filed  File Name\n', '---------------------------------------------------------------------------------------------------------------------------------------------\n']


In [3]:
ReportList = []
Company_No = []
for i in find_list:
    split_i = i.split()
    ReportList.append("https://www.sec.gov/Archives/" + split_i[-1])
    Company_No.append(split_i[-3] + "_" + split_i[-2])
    print(ReportList[:5])
    print(Company_No[:5])

['https://www.sec.gov/Archives/edgar/data/1821595/0001213900-21-018726.txt']
['1821595_2021-03-30']
['https://www.sec.gov/Archives/edgar/data/1821595/0001213900-21-018726.txt', 'https://www.sec.gov/Archives/edgar/data/1770787/0001770787-21-000009.txt']
['1821595_2021-03-30', '1770787_2021-02-26']
['https://www.sec.gov/Archives/edgar/data/1821595/0001213900-21-018726.txt', 'https://www.sec.gov/Archives/edgar/data/1770787/0001770787-21-000009.txt', 'https://www.sec.gov/Archives/edgar/data/1810140/0001213900-21-018284.txt']
['1821595_2021-03-30', '1770787_2021-02-26', '1810140_2021-03-29']
['https://www.sec.gov/Archives/edgar/data/1821595/0001213900-21-018726.txt', 'https://www.sec.gov/Archives/edgar/data/1770787/0001770787-21-000009.txt', 'https://www.sec.gov/Archives/edgar/data/1810140/0001213900-21-018284.txt', 'https://www.sec.gov/Archives/edgar/data/1751692/0001193125-21-101902.txt']
['1821595_2021-03-30', '1770787_2021-02-26', '1810140_2021-03-29', '1751692_2021-03-31']
['https://ww

In [4]:
import os
os.chdir("/Users/leftstories/Desktop/10k/testing")
  
def createfile(filename, content):
    name = filename + ".txt"  # Here we define the name of the file
    with open(name, "w") as file:
        file.write(str(content)) # Here we define its content, which will be the textual content from the 10-K files.
        file.close()
        print("Succeed!")

In [5]:
import requests
company_order = 0
unable_request = 0

for a_index in range(100):
    web_add = ReportList[a_index]
    filename = Company_No[a_index]
    
    webpage_response = requests.get(web_add, headers={'User-Agent': 'Mozilla/5.0'}) 
            # It is very important to use the header, otherwise the SEC will block the requests after the first 5.
    
    if webpage_response.status_code == 200: 
                # The HTTP 200 OK success status response code indicates that the request has succeeded. 
        body = webpage_response.content
        createfile(filename, body)
    else:
        print ("Unable to get response with Code : %d " % (webpage_response.status_code))
        unable_request += 1

        a_index +=1
		
print(unable_request) # Check to see if any of the downloads failed! Luckily, none of them failed
        # However, I strongly suggest that you check how many files the command downloaded because, more often than not, the company.idx
        # has duplicates (for instance, when the same first release the same 10-K twice for whatever reason), so the command will not
        # report an error, but it will not download all 100 files properly.

Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
Succeed!
0


In [6]:
# First, we create a list with the name of the files in the folder where we downlaoded the 10-K files.
os.chdir("/Users/leftstories/Desktop/10k/testing")
all_files = os.listdir()
# For the purpose of this exercise, I will use only 10 random files from the 100 we downloaded.
import random

# Here we basically create a list with 10 random indexes between 0 and 100 to selected the files in the folder.
random_index = []
while len(random_index) < 10:
        n = random.randint(0,10)
        if n not in random_index:
                random_index.append(n)

# Here we create a new list with the 10 chosen files.
chosen_file = [all_files[i] for i in random_index]
print(chosen_file)

['1810140_2021-03-29.txt', '1588014_2021-03-24.txt', '1580808_2021-03-08.txt', '1018840_2021-03-29.txt', '1744494_2021-03-26.txt', '1326321_2021-03-30.txt', '1769624_2021-03-26.txt', '926282_2021-02-26.txt', '894245_2021-03-30.txt', '1828972_2021-03-31.txt']


In [7]:
def read_txt(file_name):
    txt_file = open(file_name,"r",encoding='UTF8')                                       
    str_txt = txt_file.read()
    return str_txt

In [8]:
# We will use the regex module to get everything between these patterns: <DOCUMENT>\n<TYPE>10-K and </DOCUMENT>
 # Using the regex modules it quite complex, so I recommend this long video for beginners: https://www.youtube.com/watch?v=AEE9ecgLgdQ&t=1092s

import re
text_start_pattern = re.compile(r'<DOCUMENT>') 
text_end_pattern = re.compile(r'</DOCUMENT>')
type_pattern = re.compile(r'<TYPE>10-K[^\n]+')

# Here we will define a function that will be used to extract the textual data from 10-K txt files.

def textual_content(file):
    doc_start_list = [x.start() for x in text_start_pattern.finditer(file)] #assigns the first index from the starting pattern created before
    doc_end_list = [x.end() for x in text_end_pattern.finditer(file)] #assigns the last index from the ending pattern created before
    type_list = [x[len('<TYPE>'):] for x in type_pattern.findall(file)] #assigns the type of the documents, which will always be 10-K's because we restricted it before

    for doc_type, start_index, end_index in zip(type_list, doc_start_list, doc_end_list):
        report_content = file[start_index:end_index]
    return report_content

In [9]:
text_initial = read_txt(chosen_file[0])
text_10k = textual_content(text_initial)
print(text_10k[0:30])
print(text_10k[-30:])

<DOCUMENT>\n<TYPE>10-K\n<SEQUE
n</HTML>\n</TEXT>\n</DOCUMENT>


In [10]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
from bs4 import BeautifulSoup
def BeautifulSoup_clean1(str_txt):
    soup = BeautifulSoup(str_txt,'html.parser')
    return soup.get_text() # Here we return only the textual content, without the XML tags

In [11]:
import string
from nltk.tokenize import word_tokenize, sent_tokenize

def further_clean(text):
    
            # First, for each of the following characters (or symbols), the function replaces it by an empty space on the text 
    for a_sign in ['\\n', '\\t', '☐', '☒', '\xa0', '●', '“', '”']:
        text = text.replace(a_sign," ")
    
            # Here, for each punctution in a set of all existing punctuations, the function also replaces it by an empty space.
    #for a_punc in string.punctuation:
        #text = text.replace(a_punc, " ") #NOT NEEDED FOR SENT_TOKEN
    
            # Morever, the fuction replaces '\s+' (which represents a sequence of empty spaces) by an single empty space, avoiding unecessary spaces 
            # and also sets all letters to lower case to make it easier to analyse later.
    
    text = re.sub('\s+'," ", text).lower()
    
    return text.strip()
    

In [99]:
def sent_count(text):
    sent_list = sent_tokenize(text) #splits the text into words and assigns it to a new list
    total_num = len(sent_list) #count the number of words in the new list
    return sent_list

In [409]:
import pandas as pd
matchers = ['coven']
token_list = []
coven_list = []
for a_file in chosen_file:
    text = read_txt(a_file) #run the read_txt fuction that opens the txt file
    text = textual_content(text) #run the textual_content function to get the textual content from the file opened before.
    text = BeautifulSoup_clean1(text) #run the BeautifulSoup_clean1 function to clean the data even better using the html.parser algorithm. 
    text = further_clean(text) #run the further_clean function to remove symbols, punctiations, and convert strings to lower case. 
    result = sent_count(text) #run the word_count functions that tokenize the text, and deal with negative word counting.

    token_list.append(result)
    df_tokens = pd.DataFrame(token_list).T

    coven = [" ".join([result[i-1], j, result[i+1]]) for i,j in enumerate(result) if any(xs in j for xs in matchers)]
    coven_list.append(coven)
    df_covens = pd.DataFrame(coven_list).T

    
df_tokens = df_tokens.set_axis(chosen_file, axis=1, inplace=False)
df_covens = df_covens.set_axis(chosen_file, axis=1, inplace=False).T


In [410]:
coven_viols = df_covens.stack().str.extract("(\Dpurchase|item+)",expand=False).dropna()
df_coven_viols = pd.DataFrame(x, columns= ['Violated Word'])
file_name_viols = list(y.index.get_level_values(0))

df_covens['Violation'] = ''

for i in df_covens.index: 
    if i in file_name_viols: 
        df_covens['Violation'].loc[i] = 'Yes'
    else: 
        df_covens['Violation'].loc[i] = 'No'

df_covens

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Violation
1810140_2021-03-29.txt,"upon execution of the purchase agreement, aci ...",the purchase agreement contains customary repr...,the closing of the purchase agreement is subje...,see item 7 management’s discussion and analysi...,"in the event of late payment, interest shall a...","in the event of late payment, interest shall a...",the warrants also contain this beneficial owne...,the sale of additional equity securities could...,we may prepay the loan in full or in part at a...,if we terminate mr. moore’s employment without...,if we terminate mr. barry’s employment without...,"on or about march 23, 2020, the company submit...",the company may prepay the loan in full or in ...,"in the event of late payment, interest shall a...","in the event of late payment, interest shall a...",the warrants also contain this beneficial owne...,Yes
1588014_2021-03-24.txt,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No
1580808_2021-03-08.txt,if we raise additional funds through further i...,the lease term to calculate the rou asset and ...,"over the term of the 2016 credit facility, the...",None,None,None,None,None,None,None,None,None,None,None,None,None,No
1018840_2021-03-29.txt,"in addition, the following factors, categorize...",new requirements or changes in current regulat...,failure to comply with such regulations could ...,this section provides an analysis of certain c...,amortization and interest expense related to p...,the sublease agreement does not include residu...,the company is also required to pay a fee of 0...,None,None,None,None,None,None,None,None,None,Yes
1744494_2021-03-26.txt,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No
1326321_2021-03-30.txt,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No
1769624_2021-03-26.txt,"in addition, our board of directors is not cur...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No
926282_2021-02-26.txt,"to date, the company has not made any draws un...",the company agreed to certain negative covenan...,the company must be in compliance with all cov...,contingent liabilities with a fair value total...,for lease agreements entered into or reassesse...,our sales-type lease arrangements contain eith...,None,None,None,None,None,None,None,None,None,None,Yes
894245_2021-03-30.txt,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No
1828972_2021-03-31.txt,"in addition, our board of directors is not cur...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes


In [24]:
[" ".join([sentences[i-1], j, sentences[i+1]]) for i,j in enumerate(sentences) if u"covenants" in word_tokenize(j)]

['in addition, our board of directors is not currently contemplating and does not anticipate declaring any stock dividends in the foreseeable future. further, if we incur any indebtedness in connection with our initial business combination, our ability to declare dividends may be limited by restrictive covenants we may agree to in connection therewith. recent sales of unregistered securities; use of proceeds from registered securities on october 15, 2020, our sponsor purchased an aggregate of 7,187,500 founder shares for an aggregate purchase price of $25,000, or approximately $0.003 per share, in connection with our organization.']

In [30]:
matchers = ['coven','covenant', 'covenants']

coven = [" ".join([sentences[i-1], j, sentences[i+1]]) for i,j in enumerate(sentences) if any(xs in j for xs in matchers)]

['in addition, our board of directors is not currently contemplating and does not anticipate declaring any stock dividends in the foreseeable future. further, if we incur any indebtedness in connection with our initial business combination, our ability to declare dividends may be limited by restrictive covenants we may agree to in connection therewith. recent sales of unregistered securities; use of proceeds from registered securities on october 15, 2020, our sponsor purchased an aggregate of 7,187,500 founder shares for an aggregate purchase price of $25,000, or approximately $0.003 per share, in connection with our organization.']